In [1]:
import numpy as np
import cirq
import qiskit
from qiskit.circuit import qpy_serialization
from qiskit import QuantumCircuit
from typing import List
import math
import pandas as pd
from IPython.display import display
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import os
from pathlib import Path

In [3]:
import sys
sys.path.insert(0, "../lib")
from simulators import *

In [4]:
%load_ext autoreload
%autoreload 2

In [25]:
QASM_INPUT_DATA_FOLDER = "../data/QASMBench/medium/"
QASM_DATA_FOLDER = "../data/nisq_qasm"

In [6]:
qasm_file_paths = [str(p) for p in list(Path(QASM_INPUT_DATA_FOLDER).rglob("*.qasm"))]
qasm_file_paths

['../data/QASMBench/medium/ising_n10/ising_n10.qasm',
 '../data/QASMBench/medium/bb84_n8/bb84_n8.qasm',
 '../data/QASMBench/medium/seca_n11/seca_n11.qasm',
 '../data/QASMBench/medium/cc_n12/cc_n12.qasm',
 '../data/QASMBench/medium/qf21_n15/qf21_n15.qasm',
 '../data/QASMBench/medium/bv_n14/bv_n14.qasm',
 '../data/QASMBench/medium/simon_n6/simon_n6.qasm',
 '../data/QASMBench/medium/multiplier_n15/multiplier_n15.qasm',
 '../data/QASMBench/medium/vqe_uccsd_n8/vqe_uccsd_n8.qasm',
 '../data/QASMBench/medium/qaoa_n6/qaoa_n6.qasm',
 '../data/QASMBench/medium/multiply_n13/multiply_n13.qasm',
 '../data/QASMBench/medium/dnn_n8/dnn_n8.qasm',
 '../data/QASMBench/medium/qpe_n9/qpe_n9.qasm',
 '../data/QASMBench/medium/qft_n15/qft_n15.qasm',
 '../data/QASMBench/medium/vqe_uccsd_n6/vqe_uccsd_n6.qasm',
 '../data/QASMBench/medium/adder_n10/adder_n10.qasm',
 '../data/QASMBench/medium/sat_n11/sat_n11.qasm']

In [20]:
blacklist = [
    "../data/QASMBench/medium/cc_n12/cc_n12.qasm", # if not supported
    "../data/QASMBench/medium/qf21_n15/qf21_n15.qasm", # cu1
    "../data/QASMBench/medium/qpe_n9/qpe_n9.qasm", # cu1 not supported
    "../data/QASMBench/medium/vqe_uccsd_n8/vqe_uccsd_n8.qasm", # no measurement
    "../data/QASMBench/medium/vqe_uccsd_n6/vqe_uccsd_n6.qasm", # no measurement
]

In [36]:
def save(out_folder: str, metadata_dict: dict,
         df_platform_a: pd.DataFrame, df_platform_b: pd.DataFrame,
         platform_a_name: str, platform_b_name: str):
    """Save the results to a csv file."""
    # save the metadata to a json file
    assert "algo_name" in metadata_dict.keys()
    algo_name = str(metadata_dict["algo_name"])
    with open(os.path.join(out_folder, f"{algo_name}.json"), "w") as f:
        json.dump(metadata_dict, f)
    df_platform_a.to_csv(
        os.path.join(out_folder, f"{algo_name}_{platform_a_name}.csv"))
    df_platform_b.to_csv(
        os.path.join(out_folder, f"{algo_name}_{platform_b_name}.csv"))

In [45]:
N_EVALUATIONS = 100

In [47]:
platforms = [
    QiskitCircuit(), 
    CirqCircuit()
]

for circuit_number, qasm_path in enumerate(qasm_file_paths):
    print(f"Current circuit: {qasm_path}")
    qasm_str = open(qasm_path, 'r').read()
    if qasm_path in blacklist:
        print(f"Skipping ... {qasm_path}")
        continue
    
    platform_result_dfs = []
    
    # for each platform in (qiskit, cirq, etc)
    for p in platforms:
        print(p.platform_name)

        # create a circuit object form the qasm file
        p.from_qasm(qasm_str)
        #p.draw()
        
        # perform 30 evaluations [N_EVALUATION]
        data = []
        for i in range(N_EVALUATIONS):
            # run the circuit for 1024 or 8096 shots
            p.execute()
            data.append(p.get_result())

        # save the statistics for each combination to a dataframe
        df = pd.DataFrame.from_records(data)
        platform_result_dfs.append(df)

    for df, p in zip(platform_result_dfs, platforms):
        print(f"{p.platform_name} on circuit #{circuit_number}: {qasm_path}")
        display(df.head())
        
    df_qiskit, df_cirq = platform_result_dfs
    algo_name = qasm_path.split("/")[-1]
    metadata_dict = {
        "qasm_path": qasm_path,
        "algo_name": algo_name.replace(".qasm", ""),
        "n_qubits": int(re.search(r"(?<=n)\d*(?=\.)", algo_name).group(0)),
        "n_evaluations": N_EVALUATIONS,
    }

    print(f"Saving circuit: with simulation results")
    save(out_folder=QASM_DATA_FOLDER, metadata_dict=metadata_dict,
         df_platform_a=df_qiskit, df_platform_b=df_cirq,
         platform_a_name="qiskit", platform_b_name="cirq")

Current circuit: ../data/QASMBench/medium/ising_n10/ising_n10.qasm
qiskit
cirq
qiskit on circuit #0: ../data/QASMBench/medium/ising_n10/ising_n10.qasm


,0001101011,1001111010,1000111011,0101101010,1111111001,0010010101,0011001001,1100001100,1000001010,1011101010,...,1110000111,1101010110,0110101110,0001010110,1010010110,0100000101,0110111111,0110000101,0100000110,0111011101
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,5.0,NaN,2.0,1.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,NaN,7.0,1.0,1.0,5.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,2.0,NaN,1.0,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,3.0,NaN,3.0,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


cirq on circuit #0: ../data/QASMBench/medium/ising_n10/ising_n10.qasm


,0101111111,0101001101,0100101101,1111100110,0010101100,1100010011,0101110101,0001111111,0100111111,0010000111,...,1100101100,1000001100,1100010000,1000011010,1110001101,1100100111,1011011100,0000101110,1101011110,1111110000
0,7.0,8.0,2.0,3.0,5.0,1.0,2.0,4.0,2.0,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.0,2.0,3.0,4.0,NaN,NaN,NaN,5.0,NaN,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.0,7.0,2.0,1.0,2.0,NaN,NaN,4.0,3.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.0,NaN,NaN,NaN,2.0,NaN,NaN,1.0,3.0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,1.0,3.0,1.0,NaN,NaN,NaN,5.0,3.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Saving circuit: with simulation results
Current circuit: ../data/QASMBench/medium/bb84_n8/bb84_n8.qasm
qiskit
cirq
qiskit on circuit #1: ../data/QASMBench/medium/bb84_n8/bb84_n8.qasm


...

Saving circuit: with simulation results
Current circuit: ../data/QASMBench/medium/sat_n11/sat_n11.qasm
qiskit
cirq
qiskit on circuit #16: ../data/QASMBench/medium/sat_n11/sat_n11.qasm


,1000,1001,0001,1010,1111,1110,1011,0000,0111,0011,0101,1100,1101,0010,0110,0100
0,2.0,1,6.0,2,106,105,106,3.0,5.0,102,91,95,97,110,96,97
1,2.0,3,5.0,7,85,98,101,6.0,3.0,94,91,98,104,106,104,117
2,7.0,7,4.0,7,99,98,93,4.0,2.0,122,97,101,97,96,108,82
3,3.0,5,4.0,3,93,91,112,4.0,1.0,110,89,108,108,107,88,98
4,4.0,5,4.0,2,117,108,89,1.0,3.0,113,98,91,116,83,97,93


cirq on circuit #16: ../data/QASMBench/medium/sat_n11/sat_n11.qasm


,1100,1000,1010,0110,0010,0111,0011,1101,1111,1011,0101,0001,0000,1110,0100,1001
0,76,108,114,99,110,86,115,101,90,99,4.0,5.0,5.0,4.0,3.0,5.0
1,102,92,90,82,114,108,114,93,102,98,4.0,4.0,5.0,3.0,7.0,6.0
2,99,105,97,99,114,99,93,89,89,104,6.0,8.0,7.0,5.0,6.0,4.0
3,98,95,91,106,83,114,105,104,102,100,4.0,6.0,4.0,4.0,2.0,6.0
4,87,103,92,96,112,126,90,96,85,102,5.0,6.0,5.0,9.0,5.0,5.0


Saving circuit: with simulation results


In [34]:
input_string = "../data/QASMBench/medium/qft_n15/qft_n15.qasm"
match = 
print(match)

15
